In [1]:
import cv2

### Reading image, videos, webcam


In [ ]:
img = cv2.imread('resources/lena.png')
cv2.imshow('Output', img)
cv2.waitKey(0)

In [1]:
cap = cv2.VideoCapture('resources/test_video.mp4')
while True:
    success, img = cap.read()
    cv2.imshow('video', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

NameError: name 'cv2' is not defined

In [1]:
import cv2
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
while True:
    success, img = cap.read()
    cv2.imshow('video', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

### Basic functions

In [2]:
import cv2
import numpy as np

img = cv2.imread('resources/lena.png')
kernel = np.ones((5,5), np.uint8)

imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (7,7), 0)
imgCanny = cv2.Canny(img, 100, 100)
imgCanny2 = cv2.Canny(img, 150, 200)
imgDialation = cv2.dilate(imgCanny2, kernel, iterations=1)
imgEroded = cv2.erode(imgDialation, kernel)

# cv2.imshow('gray', imgGray)
# cv2.imshow('blur', imgBlur)
# cv2.imshow('canny', imgCanny)
cv2.imshow('canny2', imgCanny2)
cv2.imshow('dialation', imgDialation)
cv2.imshow('eroded', imgEroded)
cv2.waitKey(0)

32

### Resizing and cropping

In [ ]:
import cv2

img = cv2.imread('resources/lena.png')
print(img.shape)

imgResize = cv2.resize(img, (150, 150))
imgCroped = img[50:, :150]

cv2.imshow('image', img)
cv2.imshow('resize', imgResize)
cv2.imshow('crope', imgCroped)
cv2.waitKey(0)

(225, 225, 3)


### Shapes and text

In [ ]:
import cv2
import numpy as np

img = np.zeros((512, 512, 3), np.uint8)
cv2.line(img, (0, 0), (300, 300), (0, 255, 0), 3)
cv2.rectangle(img, (0, 0), (250, 350), (0, 0, 255), 2)
cv2.putText(img, 'opencv', (300, 100), cv2.FONT_ITALIC, 1, (0, 150,0 ), 1)

cv2.imshow('img', img)
cv2.waitKey(0)

### Warp perspective

In [ ]:
import cv2
import numpy as np

img = cv2.imread('resources/cards.png')

width, height = 250, 350

pts1 = np.float32([[111, 219], [287, 188], [154, 482], [352, 440]])
pts2 = np.float32([[0,0], [width,0], [0,height], [width, height]])

matrix = cv2.getPerspectiveTransform(pts1, pts2)
imgOutput = cv2.warpPerspective(img, matrix, (width, height))

cv2.imshow('img', img)
cv2.imshow('warp img', imgOutput)
cv2.waitKey(0)

### Joining images

In [ ]:
import cv2
import numpy as np
img = cv2.imread('resources/lena.png')
# img should have one depth (RGB & RGB, grayscale ...)
imgHor = np.hstack((img, img))
imgVer = np.vstack((img, img))

cv2.imshow('img hstack', imgHor)
cv2.imshow('img vstack', imgVer)
cv2.waitKey(0)

In [ ]:
import cv2
import numpy as np
from utils import stackImages

img = cv2.imread('resources/lena.png')
stackedImage = stackImages(0.5, ([img], [img])) # [img, img] to hstack
cv2.imshow('img stack', stackedImage)
cv2.waitKey(0)

### Color detection

In [ ]:
import cv2
import numpy as np
from utils import stackImages

def empty(a):
    pass

cv2.namedWindow('trackBars')
cv2.resizeWindow('trackBars', 640, 240)
cv2.createTrackbar('Hue min', 'trackBars', 0, 179, empty)
cv2.createTrackbar('Hue max', 'trackBars', 179, 179, empty)
cv2.createTrackbar('Sat min', 'trackBars', 48, 255, empty)
cv2.createTrackbar('Sat max', 'trackBars', 255, 255, empty)
cv2.createTrackbar('Val min', 'trackBars', 147, 255, empty)
cv2.createTrackbar('Val max', 'trackBars', 255, 255, empty)

while True:
    img = cv2.imread('resources/lambo.png')
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos('Hue min', 'trackBars')
    h_max = cv2.getTrackbarPos('Hue max', 'trackBars')
    s_min = cv2.getTrackbarPos('Sat min', 'trackBars')
    s_max = cv2.getTrackbarPos('Sat max', 'trackBars')
    v_min = cv2.getTrackbarPos('Val min', 'trackBars')
    v_max = cv2.getTrackbarPos('Val max', 'trackBars')

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(imgHSV, lower, upper)
    imgResult = cv2.bitwise_and(img, img, mask=mask)
        
    imgStack = stackImages(0.6, ([img, imgHSV], [mask, imgResult]))
    cv2.imshow('stecked', imgStack)
    cv2.waitKey(1)

### Contours/shape detection

In [ ]:
import cv2
import numpy as np
from utils import stackImages

def getContours(img):
    _, contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            objCor = len(approx)
            x, y, w, h = cv2.boundingRect(approx)
            
            if objCor == 3: objectType = 'Tri'
            elif objCor == 4:
                aspRatio = w/float(h)
                if aspRatio> 0.95 and aspRatio < 1.05:
                    objectType = 'Square'
                else:
                    objectType = 'Rectangle'
            elif objCor > 4: objectType = 'Circle'
            else: objectType = None
            
            cv2.rectangle(imgContour, (x,y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(imgContour, objectType,
                       (x+(w//2) + 10, y+(h//2)-10), cv2.FONT_ITALIC, 0.8,
                       (0, 0, 0), 2)
            

img = cv2.imread('resources/shapes.png')
imgContour = img.copy()

imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (7,7), 1)
imgCanny = cv2.Canny(imgBlur, 50, 50)

imgBlank = np.zeros_like(img)

getContours(imgCanny)

imgStack = stackImages(0.8, ([img, imgGray, imgBlur],
                             [imgCanny, imgContour, imgBlank]))

cv2.imshow('Stack', imgStack)
cv2.waitKey(0)

### Face detection

In [ ]:
import cv2

faceCascade = cv2.CascadeClassifier('resources/haarcascades/haarcascade_frontalface_default.xml')
img = cv2.imread('resources/lena.png')
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(imgGray, 1.1,4)

for (x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
    

cv2.imshow('Output', img)
cv2.waitKey(0)